In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

2024-03-08 08:18:09.716186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [3]:
prompt = "\n"

In [4]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids

In [5]:
from torch.utils.data import DataLoader
from datasets import load_dataset
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1")
train_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")
eval_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="test")
                

In [6]:
batch_size = 12
#test_dataset_text = [test_dataset[i : i + batch_size]["text"] for i in range(0, len(test_dataset), batch_size)]
#def batch_iterator():
#    for i in range(0, len(dataset), batch_size):
#        yield dataset[i : i + batch_size]["text"]

In [7]:
# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
embedding_size

50257

In [8]:
column_names = dataset["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)
block_size = tokenizer.model_max_length

In [9]:
from itertools import chain

# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    desc=f"Grouping texts in chunks of {block_size}",
)

train_dataset = lm_datasets["train"]
print(len(train_dataset))
eval_dataset = lm_datasets["validation"]
print(len(eval_dataset))
test_dataset = lm_datasets["test"]
print(len(test_dataset))

2318
240
274


In [25]:
import random
for index in random.sample(range(len(test_dataset)), 1):
    print(tokenizer.decode(test_dataset[index]['input_ids'])[:100])
    print(tokenizer.decode(test_dataset[index]['labels'])[:100])
    #print(f"Sample {index} of the training set: {train_dataset[index]}.")

 when the search party loses radio contact with Lieber, they must make a decision : do they sit out 
 when the search party loses radio contact with Lieber, they must make a decision : do they sit out 


In [11]:
# DataLoaders creation:
from transformers import default_data_collator

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=batch_size
)
test_dataloader = DataLoader(
    test_dataset, collate_fn=default_data_collator, batch_size=batch_size
)
print(len(train_dataloader))
print(len(eval_dataloader))
print(len(test_dataloader))

194
20
23


In [ ]:
model.compile()

In [24]:
import torch
for epoch in range(0, 1):
    active_dataloader = test_dataloader
    model.eval()
    losses = []
    for step, batch in enumerate(active_dataloader):
        #print(batch)
        if step > 2:
            break
        print(step)
        #print(batch)
        with torch.no_grad():
            outputs = model(input_ids=batch['input_ids'], labels=batch['labels'])

        loss = outputs.loss
        losses.append(loss.repeat(batch_size))
        print(loss)

    losses = torch.cat(losses)
    
import math   
try:
    eval_loss = torch.mean(losses)
    perplexity = math.exp(eval_loss)
except OverflowError:
    perplexity = float("inf")

print((f"epoch {epoch}: perplexity: {perplexity} loss: {eval_loss}"))

0
tensor(3.4247)
1
tensor(3.4102)
2
tensor(3.4637)
3
tensor(3.5491)
epoch 0: perplexity: 31.87777491065038 eval_loss: 3.461909055709839
{'perplexity': 31.87777491065038, 'eval_loss': tensor(3.4619), 'epoch': 0}


epoch 0: perplexity: 31.87777491065038 eval_loss: 3.461909055709839
{'perplexity': 31.87777491065038, 'eval_loss': tensor(3.4619), 'epoch': 0}


In [31]:
from dataclasses import dataclass
@dataclass
class DatasetSplits:
    """
        Dataclass containing the datasets for training, eval, testing, benchmark along with the name of the dataset.
        After construction, a simple type check is done with the __post_init__ hook.
    """
    train: str = None
    eval: str = None
    test: str = None
    bench: str = None

In [34]:
dataclasses = DatasetSplits()
dataclasses['train']

TypeError: 'DatasetSplits' object is not subscriptable